In [11]:
from keras.utils import np_utils, plot_model

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from keras.utils.vis_utils import model_to_dot
from IPython.display import SVG

import numpy as np
import pandas as pd

from datetime import datetime,timedelta
import matplotlib.pyplot as plt

In [12]:
def addPrefixTo(cols, prefix):
    c = []
    for col in cols:
        if col == "time":
            c.append(col)
        else:
            c.append("{0}_{1}".format(prefix, col))
    return c


def addSuffixTo(cols, suffix):
    c = []
    for col in cols:
        if col == "time":
            c.append(col)
        else:
            c.append("{0}_{1}".format(col, suffix))
    return c


def searchColumn(word, cols):
    return [col for col in cols if word in col]

# Load data

In [13]:
# 定数
# msm
lats = ["35.10", "35.15", "35.20", "35.25", "35.30", "35.35", "35.40", "35.45", "35.50"]
lons = ["139.1250", "139.1875", "139.2500", "139.3125", "139.3750", "139.4375", "139.5000", "139.5625", "139.6250"]
data_names = ["UGRD", "VGRD", "TMP", "RH", "PRMSL", "PRES", "APCP"]

# 平塚観測塔
hiratsuka_data_path = "../data/hiratsuka/avg_1h"

# 気象庁
kishocho_data_path = "../data/kishocho"
locations = ["hiratsuka", "odawara", "tsujido"]
skiprow_counts = [5, 6, 6] # csvのheader行数
colmns = [[0,1], [0,1,4,7,10,12], [0,1,4,7,10,12]]
colmn_names = [["time", "rain"], ["time", "tmp", "rain", "sun", "wind_vel", "wind_dir"], ["time", "tmp", "rain", "sun", "wind_vel", "wind_dir"]]
wind_dir = ["N", "NNE", "NE", "ENE", "E", "ESE", "SE", "SSE", "S", "SSW", "SW", "WSW", "W", "WNW", "NW", "NNW", "Q"]

step = 10
msm_first = -3
predict_hour = 1

In [14]:
def get_msm_data(year, lat, lon, data):
    csv_dir_path = "../data/csv"
    file_path="{0}/{1}/{2}".format(csv_dir_path, lon, lat)
    file_name="{0}/{1}_{2}_{3}_{4}.csv".format(file_path, lon, lat, data, year)
    data = np.genfromtxt(file_name, delimiter=",", dtype='float')
    return data[:,1]

data_msm = pd.DataFrame()
for year in range(2013, 2017 + 1):
    data_msm_year = pd.DataFrame()
    data_msm_year["time"] = pd.date_range('{0}-07-01 9:00:00'.format(year), periods=1488, freq='H')
    for lat in lats:
        for lon in lons:
            for data_name in data_names:
                data_msm_year["{0}_{1}_{2}".format(data_name, lat, lon)] = get_msm_data(year, lat, lon, data_name)
    data_msm = pd.concat([data_msm, data_msm_year])
data_msm = data_msm.reset_index(drop=True)
data_msm.columns = addPrefixTo(data_msm.columns, "msm")

In [15]:
data_hiratsuka = pd.DataFrame()
for year in range(2013, 2017 + 1):
    data_path = "{0}/{1}".format(hiratsuka_data_path, year)
    data_hiratsuka_year = pd.DataFrame(columns=["UGRD", "VGRD"])
    for month in range(7, 8 + 1):
        for day in range(1, 31 + 1):
            file_name = "{0}{1:02d}{2:02d}.csv".format(year, month, day)
            file_path = "{0}/{1}".format(data_path, file_name)

            data_day = pd.read_csv(file_path, header=0)
            data_hiratsuka_year = pd.concat([data_hiratsuka_year, data_day[["UGRD", "VGRD"]]])
            
    data_hiratsuka_year["time"] = pd.date_range('{0}-07-01 01:00:00'.format(year), periods=1488 ,freq='1h')
    data_hiratsuka = pd.concat([data_hiratsuka, data_hiratsuka_year])
data_hiratsuka = data_hiratsuka.reset_index(drop = True)
data_hiratsuka = data_hiratsuka[["time", "UGRD", "VGRD"]]
data_hiratsuka.columns = addPrefixTo(data_hiratsuka.columns, "measured")

In [16]:
wind_dir_class = pd.DataFrame([wind_dir, np.arange(0, len(wind_dir)).tolist()]).T
wind_dir_class.columns = ["wind_dir", "wind_dir_class"]

data = {}
for loc, skiprows, cols, col_name in zip(locations, skiprow_counts, colmns, colmn_names):
    df = pd.DataFrame()
    for year in range(2013, 2017 + 1):
        file_path = "{0}/{1}/{1}_{2}.csv".format(kishocho_data_path, loc, year)
        df = pd.concat([df, pd.read_csv(file_path, skiprows=skiprows, header=None)[cols]])
    df.columns = col_name
    df = df.reset_index(drop = True)
    if loc != "hiratsuka":
        df["sun"] = df["sun"].fillna(0) # 夜間日照時間をnanから0へ
        df_wind = pd.DataFrame(np_utils.to_categorical(pd.merge(df, wind_dir_class)["wind_dir_class"], 17)) # 風向きをクラスに
        df_wind.columns = wind_dir
        df = pd.concat([df, df_wind], axis=1)
        df = df.drop("wind_dir", axis=1)
    df["time"] = pd.to_datetime(df["time"])
    df.columns = addPrefixTo(df.columns, "kishocho")
    df.columns = addSuffixTo(df.columns, loc)
    data[loc] = df
dict_data_kishocho = data

In [33]:
df = pd.read_csv("../data/buoy/sagami3rd.csv", header=0)
df.columns = ["time", "buoy_tmp"]
df["time"] = pd.to_datetime(df["time"])

# 2->１時間ごとに変換
hours = (df.iloc[-1]["time"]-df.iloc[0]["time"])/timedelta(hours=1) 
_df = pd.DataFrame()
_df["time"] =  pd.date_range(df.iloc[0]["time"], periods=hours ,freq='1h')
df = pd.merge(_df, df, on="time",how='left')
df = df.fillna(method="ffill")

data_buoy = pd.DataFrame()
for year in range(2013, 2017+1):
    start = datetime(year,7,1,0,0,0)
    end = datetime(year,9,1,0,0,0)
    data_buoy = pd.concat([data_buoy, df.query('time >= \"{0}\" & time < \"{1}\"'.format(start, end))])

# Data concat

In [34]:
# # hiratsuka
# time_series_hiratsuka = pd.DataFrame(data_hiratsuka["time"])
# for i in range(0, step):
#     data_hiratsuka_ = data_hiratsuka.copy()
#     data_hiratsuka_.columns = addSuffixTo(data_hiratsuka_.columns, i)
#     data_hiratsuka_["time"] = data_hiratsuka_["time"] + timedelta(hours=i)
#     time_series_hiratsuka = pd.merge(time_series_hiratsuka, data_hiratsuka_, on="time", how="outer").sort_values(by="time")

# # MSM
# time_series_msm = pd.DataFrame(data_msm["time"])
# for i in range(msm_first, msm_first + step):
#     data_msm_ = data_msm.copy()
#     data_msm_.columns = addSuffixTo(data_msm_.columns, i)
#     data_msm_["time"] = data_msm_["time"] + timedelta(hours=i)
#     time_series_msm = pd.merge(time_series_msm, data_msm_, on="time", how="outer").sort_values(by="time")

# # 気象庁観測データ    
# data_kishocho = pd.DataFrame(dict_data_kishocho["hiratsuka"]["time"])
# for loc in locations:
#     data_kishocho = pd.merge(data_kishocho, dict_data_kishocho[loc])
    
# time_series_kishocho = pd.DataFrame(data_kishocho["time"])
# for i in range(0, step):
#     data_kishocho_ = data_kishocho.copy()
#     data_kishocho_.columns = addSuffixTo(data_kishocho_.columns, i)
#     data_kishocho_["time"] = data_kishocho_["time"] + timedelta(hours=i)
#     time_series_kishocho = pd.merge(time_series_kishocho, data_kishocho_, on="time", how="outer").sort_values(by="time")
# time_series_kishocho

# # 全体concat
# data = pd.merge(time_series_msm, time_series_hiratsuka, how='outer').sort_values(by="time")
# data = pd.merge(data, time_series_kishocho, how='outer').sort_values(by="time")
# data["time_"] = data["time"] + timedelta(hours=predict_hour)
# data_hiratsuka_ = data_hiratsuka.copy()
# data_hiratsuka_.columns = ["time_", "label_UGRD", "label_VGRD"]
# data = pd.merge(data, data_hiratsuka_, on="time_")

# data = data.dropna()
# data = data.reset_index(drop=True)
# data

In [35]:
data = pd.merge(data_hiratsuka, data_msm, on="time", how='outer').sort_values(by="time")

for loc in locations:
    data = pd.merge(data, dict_data_kishocho[loc], on="time", how='outer').sort_values(by="time")

data = pd.merge(data, data_buoy, on="time", how='outer').sort_values(by="time")

In [40]:
data.to_csv("rowdata.csv",index=False)

In [39]:
data

,time,measured_UGRD,measured_VGRD,msm_UGRD_35.10_139.1250,msm_VGRD_35.10_139.1250,msm_TMP_35.10_139.1250,msm_RH_35.10_139.1250,msm_PRMSL_35.10_139.1250,msm_PRES_35.10_139.1250,msm_APCP_35.10_139.1250,...,kishocho_S_tsujido,kishocho_SSW_tsujido,kishocho_SW_tsujido,kishocho_WSW_tsujido,kishocho_W_tsujido,kishocho_WNW_tsujido,kishocho_NW_tsujido,kishocho_NNW_tsujido,kishocho_Q_tsujido,buoy_tmp
7481,2013-07-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.06
0,2013-07-01 01:00:00,0.891521,5.549878,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.06
1,2013-07-01 02:00:00,1.202461,5.350940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.90
2,2013-07-01 03:00:00,1.273486,5.114210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.90
3,2013-07-01 04:00:00,1.565447,4.831372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.82
4,2013-07-01 05:00:00,1.703561,4.048579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.82
5,2013-07-01 06:00:00,2.147069,2.927265,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.02
6,2013-07-01 07:00:00,2.215773,2.555584,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.02
7,2013-07-01 08:00:00,1.371071,1.143301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.82
8,2013-07-01 09:00:00,1.876642,0.650377,-2.294930,2.158230,295.461,80.5930,101631.0,100825.0,0.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.82
